In [1]:
import random
import numpy as np
import tensorflow as tf
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
import unidecode

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6120)])
#   except RuntimeError as e:
#     print(e)

In [3]:
lemmatizer=WordNetLemmatizer()
data = open("Json_detalle_productos_bot.json").read()
intents = json.loads(data)
words = pickle.load(open('words_chatbot_productos.pkl','rb'))
clases = pickle.load(open('clases_chatbot_productos.pkl','rb'))
model = tf.keras.models.load_model('Chatbot_base_chatbot_producto_v2.tf')

In [4]:
print(intents['intents'])

[{'tag': 'Back-Test', 'patterns': ['back', 'retrospectivas', 'retrospectiva'], 'responses': ['- Aplicación de Back-Testing  a Modelos de estimación del Valor en Riesgo, para verificar su correcto ajuste a la Historia de la EIF.- Se recomienda Aplicar Backtest a cada Modelo propuesto.']}, {'tag': 'CLTV Based Pricing', 'patterns': ['cltv', 'lifetime'], 'responses': ['- Modelo de Precios en base al Valor del Cliente en el Tiempo, con un enfoque Estadístico Clásico - Comportamental, que persigue un ajuste del valor de cliente en función del retorno que genera hacia la Institución Financiera.- Funciona como complemento a la visión Riesgo Céntrica del Pricing. - Mediante la aplicación conjunta de los modelos de Pricing se consigue un ajuste Dinámico del Pricing.']}, {'tag': 'Atrittion Score', 'patterns': ['atrittion', 'atrittion'], 'responses': ['- Modelo de Retención, basado en Estadística Clásica, para predecir la Probabilidad de Deserción de Clientes.- Permite generar estrategías de reten

In [5]:
model.summary()

Model: "chatbot_trainer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  8448      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  8256      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  1300      
                                                                 
Total params: 18,004
Trainable params: 18,004
Non-t

In [6]:
# model.predict([])

In [7]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize((unidecode.unidecode(word)).lower())
                      for word in sentence_words]
    return sentence_words



In [8]:
# this function will return 1 if the word is contained in our sentence and in the list previously defined 
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [9]:
# This function predicts a cleansed data with a mark in the bow (flag = 1 )
# It also appends the class - tag in the clases list where it belongs 
def predict_clases(sentence,model):
    print(clean_up_sentences(sentence))
    p = bow(sentence,words,show_details=True)
    print(p)
    res = model.predict(np.array([p]))[0]
    
    ERROR_THRESHOLD=0.3#0.25
    results = [[ i , r ] for i , r  in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key = lambda x : x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({
            'intent' : clases[r[0]],
            'probability': str(r[1])

        } )
        return return_list


In [10]:
# this function will print a random response in the class according the input that the user made
def get_response(intents_list,intents_json):
    print('intents_list',intents_list)
    tag = intents_list[0]['intent']
    list_of_intents = intents_json ['intents']
    result = ""
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print ("Esta vivo!")

Esta vivo!


In [11]:


# while True:
#     message = ""
#     ints = predict_clases(message)
#     res = get_response(ints , intents)
#     print(res)

#     # if message == 'salir':
    #     C=False


In [12]:
# message = "whats your name?"

# ints = predict_clases(message,model)
# res = get_response(ints , intents)
# print(res)

In [16]:
message = "lgd"
print("esto es message",message)
print("esto es message",model)

ints = predict_clases(message,model)
print("esto es ints",ints)
res = get_response(ints , intents)
print(res)

esto es message lgd
esto es message <keras.saving.legacy.saved_model.load.Chatbot_trainer object at 0x7fe10a6a8850>
['lgd']
found in bag: lgd
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 18ms/step
esto es ints [{'intent': 'Loss Given By Default', 'probability': '1.0'}]
intents_list [{'intent': 'Loss Given By Default', 'probability': '1.0'}]
- Estimación de Pérdidas Financieras dado el Incumplimiento de Préstamos, en función de Modelos Estadísticos Clásicos.- Forma parte de la Metodología de Basilea, para Cálculo de Pérdidas Esperadas y su consecuente requerimiento de Capital Económico.
